# Demo 2: Training a Neural Network

* Using PyTorch
* Expected time: 5 minutes


Let's recall together what we need to train a NN:
* NN
* Input/output representation
* Data
* Differentiable programming
* Loss function
* Optimization algorithm

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import MNIST
from tqdm import tqdm
from typing import List
torch.manual_seed(0)  # for reproducibility

## Step 1: Data

For this demo, we will use the MNIST digit classification data.

In [ ]:
dataset = MNIST("mnist", download=True)

It's always a good idea to visualize and analyze the data to gain some understanding of what's going on.

In [ ]:
# How many images are there in the dataset?
len(dataset)

In [ ]:
# What is the format of the data?
dataset[0]

It's a tuple of two elements. In Python, it's easy enough to unpack:

In [ ]:
input, label = dataset[0]

In [ ]:
# What does each "example" look like?
print(np.array(input))
print(np.array(input).shape)

In [ ]:
plt.figure()
plt.title(f"label: {label}")
plt.imshow(np.array(input), cmap="gray")
plt.show()

Aha, the handwriting of the digit "5" corresponds to the label "5", so we have 60000 grayscale images whose sizes are 28x28 and the corresponding numerical labels.

## Step 2: Input/Output Representation

For our model to process the image, it's more convenient to "flatten" the images to be a long vector instead of a 28x28 matrix.

In [ ]:
input_reshaped = np.array(input).reshape(-1)

In [ ]:
print(input_reshaped)
print(input_reshaped.shape)

So instead of 28x28 images, we are working with 784-dimensional vectors. (28x28 = 784)

As for the output representation, remember that we had 1000-dimensional output for our ImageNet demo earlier. Now, we have the digit classification problem, meaning...

Q: What would our predictions look like, when the neural network produces it?
* A. "5" or something like that
* B. An image of the digit 5
* C. A 10-dimensional array

## Step 3: Differentiable Programming

Libraries like PyTorch, TensorFlow, Caffe, MXNet and more allow us to simplify training neural networks with more interpretable code. Differentiable programming libraries usually work with "tensors" which are basically the same as arrays, but they have a special property where differentiable operations are recorded into the variables. Easier to show this than tell:

In [ ]:
input_tensor = torch.tensor(input_reshaped)

Q: What do you think input_tensor looks like?
* A. Basically same as input_reshaped
* B. Different from input_reshaped
* C. None of the above

In [ ]:
print(input_tensor)
print(input_tensor.shape)

Now, we will instantiate our neural network. In PyTorch's language, producing predictions from a neural network (model) is a differentiable operation, which we will see soon.

## Step 4: Neural Network

In this step, we will instantiate a neural network using PyTorch's `nn.Sequential` class. `nn.Sequential` lets us express a sequence of differentiable operations quite gracefully.

In [ ]:
model = nn.Sequential(
    nn.Linear(784, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 10),
)

## Pop Quiz!

Q: What do you think `nn.Linear(784, 16)` is about?

* A. a matrix multiplication that reduces a 784-dimensional vector to a 16-dimensional vector
* B. a linear interpolation between a 784-dimensional vector and a 16-dimensional vector
* C. none of the above

Let's see what the output looks like when we put our 784-dimensional tensor through.

In [ ]:
# forward() is basically the same as predict(), 
# but it's says more explicitly that the operation is differentiable.
# the term "forward" comes from "feedforward" in backpropagation.
pred_tensor = model.forward(input_tensor)

Oops, looks like the data types are incompatible. Here come some dull/cryptic bits when working with differentiable programming libraries: you must make sure that the data types and input shapes are consistent.

In [ ]:
pred_tensor = model.forward(input_tensor.float())

Whew! So we needed to massage our input image as follows:
* Turn the image into a 28x28 array
* Turn the 28x28 array in to a 784-dimensional array
* Turn the array in to a tensor
* Cast the data type of the tensor to float to be consistent with expected data type

Let's look at the shape of `pred_tensor`:

In [ ]:
print(pred_tensor.shape)

Q: Is there any difference from what you were expecting?

Now, let's look at what's inside `pred_tensor`.

Q: Do you think `pred_tensor` would be a "correct" prediction?

In [ ]:
print(pred_tensor)

In [ ]:
print(torch.argmax(pred_tensor))

Of course not! Since the network is completely untrained and it has no idea what a digit 5 looks like.

## Step 5: Loss Function

As you see, initializing a NN and just producing a prediction doesn't do anything. We must provide a "training signal" to the network using a loss function, as discussed in the lecture.

In [ ]:
label_tensor = torch.tensor(label)
print(torch.argmax(pred_tensor), label_tensor)

So right now the neural network thinks the image is a "3". Of course, it has no idea what it's saying. Let's give it a feedback of a sort, expressed via a *loss function*.

In [ ]:
# We won't go into details about what CrossEntropyLoss is, but just know that this is the correct loss
# for our task and current format of input/output representations
cross_entropy_loss = nn.CrossEntropyLoss()

In [ ]:
# Another shape massaging, but I won't get into it too much. Once you wrangle tensors for long enough, 
# it becomes pretty clear.
loss = cross_entropy_loss.forward(pred_tensor.unsqueeze(0), label_tensor.unsqueeze(0))
print(loss)

Aha, now we successfully quantified the performance of our current NN.

Q: Making an NN "better" would correspond to...
* A. Minimizing the value coming from `cross_entropy_loss.forward()`
* B. Maximizing the value coming from `cross_entropy_loss.forward()`
* C. None of the above


## Step 6: Optimization algorithm

The very last step for us is to use an optimization algorithm to optimize the loss value. How? Well, it's a pretty complex concept. For now, I'll present you PyTorch's beautifully elegant way to use optimization algorithms out-of-the-box.

In [ ]:
optimizer = torch.optim.Adam(lr=3e-4, params=model.parameters())

Explanation:
* "Adam" is a widely popular optimization algorithm that has 10k+ citations. It's not perfect, but it's a great starting point.
* `lr` stands for learning rate. For now, I'm using some number according to my intuitive guess of what a good learning rate looks like.
* `params` specifies what I'm changing in order to make the loss function lower, hence the connections of our NN.

Optimizers are easy to use, if you forgive some arbitrary-looking steps.

In [ ]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

Just for the demonstration of what that chunk of code just did, I'll redo the prediction and loss calculation steps.

In [ ]:
pred_tensor = model.forward(input_tensor.float().reshape(1, -1))
loss = cross_entropy_loss.forward(pred_tensor, label_tensor.reshape(-1))
print(loss)

Q: Note the differences in number. What do you think happened?
* A. The parameters of our NN got changed by optimizer
* B. The prediction from our NN got changed
* C. All of the above

If I did this many many times, my NN would know what the correct label is!

In [ ]:
for _ in tqdm(range(1000)):
    # get the latest version of prediction
    pred_tensor = model.forward(input_tensor.float().reshape(1, -1))
    # compute loss based on predictions (note that label_tensor never changes!)
    loss = cross_entropy_loss.forward(pred_tensor, label_tensor.reshape(-1))
    # perform an optimization step, which mutates our NN
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # rinse and repeat 1000 times!

In [ ]:
print(loss)

Wow, that's a pretty "small" number. Here's what our latest prediction looks like:

In [ ]:
pred_tensor = model.forward(input_tensor.float().reshape(1, -1))
print(pred_tensor)
print(torch.argmax(pred_tensor), label_tensor.reshape(-1))

That's a huge improvement from thinking it's a "3"!

## Putting it altogether

Thus far, for the sake of demonstration, we only used one example (image/label pair) to train our neural network. Of course, this is not super useful. For example, it never had the chance to see what other digits look like!

Ideally, we should use all of the 60000 digits to train a decent digit classifier. For the interest of time, though, we use only 100 digits for now. Forgive a few magic lines that I don't have time to get into, but believe that all of the 6 steps discussed above are still present.

In [ ]:
def collate(examples):
    inputs = torch.stack([torch.tensor(np.array(input)).reshape(-1) for input, _ in examples], dim=0)
    labels = torch.tensor([label for _, label in examples])
    return inputs, labels    

subset = Subset(dataset, np.arange(100))
data_loader = DataLoader(subset, batch_size=100, collate_fn=collate)

model = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
)
optimizer = torch.optim.Adam(lr=3e-4, params=model.parameters())

Q: Look at the way we instantiated `model` just now. Why would I want to use 256 as the layer size now?

In [ ]:
for _ in tqdm(range(100)):
    epoch_loss = 0.
    for example in data_loader:
        input, label = example
        input_tensor = input.float()
        label_tensor = label.long()
        
        # gpu acceleration
        model = model.cuda()
        input_tensor = input_tensor.cuda()
        label_tensor = label_tensor.cuda()
        
        # get the latest version of prediction
        pred_tensor = model.forward(input_tensor / 255.)

        # compute loss based on predictions (note that label_tensor never changes!)
        loss = cross_entropy_loss.forward(pred_tensor, label_tensor)
        
        # perform an optimization step, which mutates our NN
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.data
    epoch_loss /= len(subset)

In [ ]:
# for our case, inference doesn't need gpu acceleration
model = model.cpu()
input_tensor = input_tensor.cpu()
label_tensor = label_tensor.cpu()

In [ ]:
for i in range(10):
    input, label = subset[i]
    input_array = np.array(input)
    input_reshaped = np.array(input).reshape(1, -1)
    input_tensor = torch.tensor(input_reshaped).float() / 255.
    pred_tensor = model.forward(input_tensor)
    prob_tensor = torch.softmax(pred_tensor, dim=1)
    
    plt.figure()
    plt.imshow(input_array, cmap="gray")
    plt.title(f"prediction:{torch.argmax(prob_tensor).item()}")
    plt.show()

# But Before You Go...

Is our NN actually useful in the wild? i.e. if I show more digit images that were NOT used during training, what happens?

In [ ]:
for i in np.arange(100, 111):
    input, label = dataset[i]
    input_array = np.array(input)
    input_reshaped = np.array(input).reshape(1, -1)
    input_tensor = torch.tensor(input_reshaped).float() / 255.
    pred_tensor = model.forward(input_tensor)
    prob_tensor = torch.softmax(pred_tensor, dim=1)
    
    plt.figure()
    plt.imshow(input_array, cmap="gray")
    plt.title(f"prediction:{torch.argmax(prob_tensor).item()}")
#     print(torch.argmax(pred_tensor), label)

Lots of errors start showing up. This is called *overfitting*. We won't be getting too much in detail about this, but be aware that for your NN to be useful at inference time, you need more "general" performance, usually achieved by e.g.:
* Using more data
* Using sparser architectures
* Using augmentations
* Using better *hyperparameters*